In [1]:
import pandas as pd

df=pd.read_csv('new12.csv')

In [2]:
df = df.sample(frac=0.3)
df.head()

,Unnamed: 0,num,name,content
19033,19033,9257755,777.charlie.(2022).eng.1cd,ï»¿ support us and become vip member to remove...
34173,34173,9317293,elite.s06.e07.mascaras.(2022).eng.1cd,ï»¿ emergency use the free code joinnow at â¨...
18208,18208,9254734,survivor.s25.e14.million.dollar.question.(2012...,">> jeff: survivor it's toughest, and most dan..."
75130,75130,9490657,forgiveness.(2021).eng.1cd,ï»¿ [muffled yelling] [man] you fucking bitch!...
4428,4428,9200853,la.brea.s01.e01.pilot.(2021).eng.1cd,"ï»¿ take a look outside it's a beautiful day, ..."


In [3]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [4]:
import re

def clean_text(text):
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
    cleaned_text = re.sub(r'[^\x00-\x7F]+', '', cleaned_text)
    cleaned_text = re.sub(r'\d{2}:\d{2}:\d{2}\d{3} --> \d{2}:\d{2}:\d{2},\d{3}', '', cleaned_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    cleaned_text = cleaned_text.lower().strip()
    
    return cleaned_text

from tqdm import tqdm
tqdm.pandas()

df['content'] = df['content'].progress_apply(clean_text)

100%|███████████████████████████████████████████████████████████████████████████| 24749/24749 [01:22<00:00, 301.43it/s]


In [5]:
def chunk_text(text, chunk_size=500, overlap=50):
    chunks = []
    words = text.split()
    for start in range(0, len(words), chunk_size - overlap):
        chunks.append(' '.join(words[start:start + chunk_size]))
    return chunks

df['chunked_text'] = df['content'].progress_apply(lambda x: chunk_text(x))

100%|██████████████████████████████████████████████████████████████████████████| 24749/24749 [00:06<00:00, 3815.17it/s]


In [6]:
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



     ---------------------------------------- 0.0/137.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/137.6 kB ? eta -:--:--
     -------- ------------------------------ 30.7/137.6 kB 1.3 MB/s eta 0:00:01
     ------------------------------------ - 133.1/137.6 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 137.6/137.6 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/171.5 kB ? eta -:--:--
   ---------------------------------------- 171.5/171.5 kB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/388.9 kB ? eta -:--:--
   ----------------------- ---------------- 225.3/388.9 kB 6.9 MB/s eta 0:00:01
   ----------------------------------- ---- 348.2/388.9 kB 7.2 MB/s eta 0:00:01
   ---------------------------------------- 388.9/388.9 kB 4.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/198.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/198.5 MB 6.8 MB/s eta 0:00:30
  

In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\Lenovo\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
df["chunk_embeddings"] = df["chunked_text"].progress_apply(lambda x: model.encode(x))

100%|██████████████████████████████████████████████████████████████████████████| 24749/24749 [7:42:55<00:00,  1.12s/it]


In [9]:
df['chunk_embeddings']
df.to_csv('embedded_df_30_percent.csv', index=False)